# Cleaning costing data
### date 6/7/2024

In [55]:
import pandas as pd
import xlwings as xw
import os
import re
from datetime import datetime


In [56]:
# Load the data
df_raw = pd.read_csv("../qualtrics_survey/SITT-MAT Weekly Implementation Activity Survey for SITT-MAT Team_June 10, 2024_11.39.csv")
# remove first two rows
df_raw = df_raw.iloc[2:]


## Functions for cleaning

In [57]:
# Function to extract time and convert to hours
def extract_hours(task):
    minutes = re.findall(r'(\d+)\s*min', task)
    if minutes:
        return int(minutes[0]) / 60
    hours = re.findall(r'(\d+)\s*hour', task)
    if hours:
        return int(hours[0])
    other_minutes = re.findall(r'(\d+)\s*minutes', task)
    if other_minutes:
        return int(other_minutes[0]) / 60
    return 0

# Function to check if the "Notes" column contains minutes
def contains_minutes(notes):
    return bool(re.search(r'\d+\s*(min|minutes)', notes))


In [58]:
def rename_columns(df):
    rename_dict = {
    'Q1': 'Name',
    'Q2': 'Start Date',
    'Q3': 'End Date',
    'Q6_1_1': 'EMF_meetings_mins',
    'Q6_1_2': 'EMF_meetings_notes',
    'Q6_2_1': 'EMF_datacollection_mins',
    'Q6_2_2': 'EMF_datacollection_notes',
    'Q6_3_1': 'EMF_REDCap_mins',
    'Q6_3_2': 'EMF_REDCap_notes',
    'Q6_4_1': 'EMF_dashboard_mins',
    'Q6_4_2': 'EMF_dashboard_notes',
    'Q6_5_1': 'EMF_reporttemplate_mins',
    'Q6_5_2': 'EMF_reporttemplate_notes',
    'Q6_6_1': 'EMF_IMATcalculations_mins',
    'Q6_6_2': 'EMF_IMATcalculations_notes',
    'Q6_7_1': 'EMF_datawebinar_prep_mins',
    'Q6_7_2': 'EMF_datawebinar_prep_notes',
    'Q6_8_1': 'EMF_datawebinar_invite_mins',
    'Q6_8_2': 'EMF_datawebinar_invite_notes',
    'Q6_9_1': 'EMF_datawebinar_host_mins',
    'Q6_9_2': 'EMF_datawebinar_host_notes',
    'Q6_10_1': 'EMF_datainquiries_mins',
    'Q6_10_2': 'EMF_datainquiries_notes',
    'Q6_11_1': 'EMF_ServiceRegistry_inforequest_mins',
    'Q6_11_2': 'EMF_ServiceRegistry_inforequest_notes',
    'Q6_12_1': 'EMF_ServiceRegistry_validation_mins',
    'Q6_12_2': 'EMF_ServiceRegistry_validation_notes',
    'Q6_13_1': 'EMF_ServiceRegistry_analysis_mins',
    'Q6_13_2': 'EMF_ServiceRegistry_analysis_notes',
    'Q6_14_1': 'EMF_generatereport_mins',
    'Q6_14_2': 'EMF_generatereport_notes',
    'Q6_15_1': 'EMF_returnreport_mins',
    'Q6_15_2': 'EMF_returnreport_notes',
    'Q7_1_1': 'NIATxMAT_meetings_mins',
    'Q7_1_2': 'NIATxMAT_meetings_notes',
    'Q7_2_1': 'NIATxMAT_academyplan_mins',
    'Q7_2_2': 'NIATxMAT_academyplan_notes',
    'Q7_3_1': 'NIATxMAT_academyinvites_mins',
    'Q7_3_2': 'NIATxMAT_academyinvites_notes',
    'Q7_4_1': 'NIATxMAT_academyparticipation_mins',
    'Q7_4_2': 'NIATxMAT_academyparticipation_notes',
    'Q8_1_1': 'InternalF_meetings_mins',
    'Q8_1_2': 'InternalF_meetings_notes',
    'Q8_2_1': 'InternalF_content_mins',
    'Q8_2_2': 'InternalF_content_notes',
    'Q8_3_1': 'InternalF_datacollection_mins',
    'Q8_3_2': 'InternalF_datacollection_notes',
    'Q8_4_1': 'InternalF_invites_mins',
    'Q8_4_2': 'InternalF_invites_notes',
    'Q8_5_1': 'InternalF_identifiedchangeleader_mins',
    'Q8_5_2': 'InternalF_identifiedchangeleader_notes',
    'Q8_6_1': 'InternalF_trackedsiteparticipation_mins',
    'Q8_6_2': 'InternalF_trackedsiteparticipation_notes',
    'Q8_7_1': 'InternalF_ledcoachcalls_mins',
    'Q8_7_2': 'InternalF_ledcoachcalls_notes',
    'Q8_8_1': 'InternalF_additionalsupport_mins',
    'Q8_8_2': 'InternalF_additionalsupport_notes',
    'Q9_1_1': 'ExternalF_meetings_mins',
    'Q9_1_2': 'ExternalF_meetings_notes',
    'Q9_2_1': 'ExternalF_content_mins',
    'Q9_2_2': 'ExternalF_content_notes',
    'Q9_3_1': 'ExternalF_datasystems_mins',
    'Q9_3_2': 'ExternalF_datasystems_notes',
    'Q9_4_1': 'ExternalF_participantinvites_mins',
    'Q9_4_2': 'ExternalF_participantinvites_notes',
    'Q9_5_1': 'ExternalF_trackedsiteparticipation_mins',
    'Q9_5_2': 'ExternalF_trackedsiteparticipation_notes',
    'Q9_6_1': 'ExternalF_plannedcoachcalls_mins',
    'Q9_6_2': 'ExternalF_plannedcoachcalls_notes',
    'Q9_7_1': 'ExternalF_ledcoachcalls_mins',
    'Q9_7_2': 'ExternalF_ledcoachcalls_notes',
    'Q9_8_1': 'ExternalF_additionalsupport_mins',
    'Q9_8_2': 'ExternalF_additionalsupport_notes',
    'Q44_Id': 'ExternalF_coachtrackerid',
    'Q44_Name': 'ExternalF_coachtrackername',
    'Q44_Size': 'ExternalF_coachtrackersize',
    'Q44_Type': 'ExternalF_coachtrackertype',
    'Q10_1_1': 'SIC/COINS_meetings_mins',
    'Q10_1_2': 'SIC/COINS_meetings_notes',
    'Q10_2_1': 'SIC/COINS_content_mins',
    'Q10_2_2': 'SIC/COINS_content_notes',
    'Q10_3_1': 'SIC/COINS_trackedinfo_mins',
    'Q10_3_2': 'SIC/COINS_trackedinfo_notes',
    'Q10_4_1': 'SIC/COINS_fidelityscale_mins',
    'Q10_4_2': 'SIC/COINS_fidelityscale_notes',
    'Q10_5_1': 'SIC/COINS_sustainementscale_mins',
    'Q10_5_2': 'SIC/COINS_sustainementscale_notes',
    'Q10_6_1': 'SIC/COINS_developedtracker_mins',
    'Q10_6_2': 'SIC/COINS_developedtracker_notes',
    'Q10_7_1': 'SIC/COINS_validation_mins',
    'Q10_7_2': 'SIC/COINS_validation_notes',
    'Q10_8_1': 'SIC/COINS_data_mins',
    'Q10_8_2': 'SIC/COINS_data_notes',
    'Q11_1_1': 'SITMAATTeam_meetings_mins',
    'Q11_1_2': 'SITMAATTeam_meetings_notes',
    'Q11_2_1': 'SITMAATTeam_costsurvey_mins',
    'Q11_2_2': 'SITMAATTeam_costsurvey_notes',
    'Q11_3_1': 'SITMAATTeam_validatecostsurvey_mins',
    'Q11_3_2': 'SITMAATTeam_validatecostsurvey_notes',
    'Q11_4_1': 'SITMAATTeam_costdata_mins',
    'Q11_4_2': 'SITMAATTeam_costdata_notes',
    'Q12_1': 'additionalactivity_notes1',
    'Q12_2': 'additionalactivity_notes2',
    'Q12_3': 'additionalactivity_notes3',
    'Q12_4': 'additionalactivity_notes4',
    'Q12_5': 'additionalactivity_notes5',
    'Q13': 'additionalcosting_notes'
}
     # Identify extra columns
    extra_columns = [col for col in df.columns if col not in rename_dict]
    if extra_columns:
        print(f"Warning: The following columns are not renamed and will be dropped: {extra_columns}")
    

    return df.rename(columns=rename_dict)

In [59]:

def clean_additional_activities(df_raw):
    # Goal: name, start date, end date, category, activity details, mins, hrs
    sittmaat_weekex = df_raw
    sittmaat_weekex = rename_columns(sittmaat_weekex)

    # Subset df without metadata (first 2 rows) and only questions related to "not involved in SITTMAT"
    sittmaat_weekex = sittmaat_weekex[["Name", "Start Date", "End Date", "additionalactivity_notes1", "additionalactivity_notes2", "additionalactivity_notes3",
                                      "additionalactivity_notes4", "additionalactivity_notes5", "additionalcosting_notes"]]

    # Renaming columns
    sittmaat_weekex.columns = [
        "Name", "Start Date", "End Date", "Additionalactivity1_notes", "Additionalactivity2_notes", 
        "Additionalactivity3_notes", "Additionalactivity4_notes", "Additionalactivity5_notes", "Additionalcosting_notes"
    ]

    # Wide to long form (Category/Activity/EstMin)
    sittmaat_weekex = pd.melt(sittmaat_weekex, id_vars=["Name", "Start Date", "End Date"], 
                              var_name="Activity", value_name="Notes")

    # Omit NA rows
    sittmaat_weekex = sittmaat_weekex.dropna(subset=['Notes'])

    # Add activity category column
    sittmaat_weekex['Category'] = sittmaat_weekex['Activity'].apply(lambda x: "Additional Activities" if x.startswith("Additionalactivity") else "Additional Costing" if x.startswith("Additionalcosting") else None)

    # Remove Activity column
    sittmaat_cleaned = sittmaat_weekex.drop(columns=["Activity"])
    
    # Split into hours and no hours based on the presence of minutes in the "Notes" column
    sittmaat_cleaned_hours = sittmaat_cleaned[sittmaat_cleaned['Notes'].apply(contains_minutes)]
    sittmaat_cleaned_no_hours = sittmaat_cleaned[~sittmaat_cleaned['Notes'].apply(contains_minutes)]

    # Get the hours from the minutes indicated
    sittmaat_cleaned_hours.insert(4, 'hours', sittmaat_cleaned_hours['Notes'].apply(extract_hours))

    return sittmaat_cleaned_hours, sittmaat_cleaned_no_hours


In [60]:
def clean_not_involved(df_raw):
    # Subset the dataframe where Q4 equals the specific string
    sittmaat_weekex = df_raw[df_raw['Q4'] == "No (Note: selecting this will take you to the end of survey)"]
    # Subset with only specific columns
    sittmaat_not_involved = sittmaat_weekex[['Q1', 'Q2', 'Q3']]

    # Renaming columns
    sittmaat_not_involved.columns = ["Name", "Start Date", "End Date"]
    return sittmaat_not_involved

In [61]:
def clean_optional_notes(df_raw):
    # Filter the dataframe where Q4 equals "Yes"
    sittmaat_weekex = df_raw[df_raw['Q4'] == "Yes"]

    # Drop the initial metadata columns (first 17 columns)
    sittmaat_weekex = sittmaat_weekex.drop(columns=sittmaat_weekex.columns[:17])

    # Drop the fourth column (index 3 after the above operations)
    sittmaat_weekex = sittmaat_weekex.drop(columns=['Q4'])
    
    #rename columns
    sittmaat_weekex = rename_columns(sittmaat_weekex)

    
    # pivot longer
    sittmaat_weekex = sittmaat_weekex.melt(id_vars=["Name", "Start Date", "End Date"], 
                                           value_vars=[col for col in sittmaat_weekex.columns if col.endswith('_mins') or col.endswith('_notes')],
                                           var_name="Activity", value_name="Notes")


    sittmaat_weekex[['Activity', 'ValueType']] = sittmaat_weekex['Activity'].str.extract(r'(.+)_([a-z]+)')
    sittmaat_weekex = sittmaat_weekex.drop_duplicates()
    sittmaat_weekex = sittmaat_weekex.pivot(index=['Name', "Start Date", "End Date", 'Activity'], 
                                            columns='ValueType', values='Notes').reset_index()

    # rename new columns
    sittmaat_weekex = sittmaat_weekex.rename(columns={
        'mins': 'Mins',
        'notes': 'Optional Notes'
    })
    
    # Omit NA rows
    sittmaat_weekex = sittmaat_weekex.dropna(subset=['Mins'])

    # Convert mins to numeric and create hours
    sittmaat_weekex['Mins'] = pd.to_numeric(sittmaat_weekex['Mins'])
    sittmaat_weekex['Hrs'] = sittmaat_weekex['Mins'] / 60

    # Add activity category column
    sittmaat_weekex['Category'] = sittmaat_weekex['Activity'].apply(lambda x: 
        "EMF" if x.startswith("EMF_") else 
        "NIATxMAT Academy" if x.startswith("NIATxMAT_") else 
        "Internal Facilitation" if x.startswith("InternalF_") else 
        "External Facilitation" if x.startswith("ExternalF_") else 
        "SIC/COINS" if x.startswith("SIC/COINS_") else 
        "SITMAAT Purveyor Team" if x.startswith("SITMAATTeam_") else 
        "Additional Relevant Activities" if x.startswith("additionalactivity") else 
        "Additional Costing Tasks" if x.startswith("additionalcosting_") else 
        None
    )
    
    sittmaat_weekex = sittmaat_weekex.reset_index(drop=True)
    return sittmaat_weekex

In [62]:
def clean_involved(df_raw):
    raw_sittmaat_weekex = df_raw
    # Drop metadata and the first two rows
    sittmaat_weekex = raw_sittmaat_weekex.drop(columns=raw_sittmaat_weekex.columns[:17])
    # Subset data where Q4 == "Yes"
    sittmaat_weekex = sittmaat_weekex[sittmaat_weekex['Q4'] == "Yes"]

    # Drop the Q4 column
    sittmaat_weekex = sittmaat_weekex.drop(columns=['Q4'])


    #rename columns
    sittmaat_weekex = rename_columns(sittmaat_weekex)

    # Convert the DataFrame from wide to long format
    sittmaat_weekex_long = pd.melt(sittmaat_weekex, id_vars=['Name', 'Start Date', 'End Date'],
                                   value_vars=[col for col in sittmaat_weekex.columns if col.endswith('_mins')],
                                   var_name='Activity', value_name='Mins')


    # Remove rows with NaN values in the 'Mins' column
    sittmaat_weekex_long = sittmaat_weekex_long.dropna(subset=['Mins'])

    # Convert 'Mins' to numeric and calculate 'Hrs'
    sittmaat_weekex_long['Mins'] = pd.to_numeric(sittmaat_weekex_long['Mins'])
    sittmaat_weekex_long['Hrs'] = sittmaat_weekex_long['Mins'] / 60

    # Add 'Category' column
    sittmaat_weekex_long['Category'] = sittmaat_weekex_long['Activity'].apply(lambda x: 
        'EMF' if x.startswith('EMF_') else
        'NIATxMAT Academy' if x.startswith('NIATxMAT_') else
        'Internal Facilitation' if x.startswith('InternalF_') else
        'External Facilitation' if x.startswith('ExternalF_') else
        'SIC/COINS' if x.startswith('SIC/COINS_') else
        'SITMAAT Purveyor Team' if x.startswith('SITMAATTeam_') else
        'Additional Relevant Activities' if x.startswith('additionalactivity') else
        'Additional Costing Tasks' if x.startswith('additionalcosting_') else
        None
    )

    return sittmaat_weekex_long


## Putting sheets into an excel

In [63]:
# Function to export DataFrames to an Excel file with multiple sheets
def export_to_excel(file_name, dfs, sheet_names):
    full_path = os.path.abspath(file_name)
    
    # Check if the file already exists and delete it if it does
    if os.path.exists(full_path):
        os.remove(full_path)
    
    with xw.App(visible=False) as app:
        wb = app.books.add()

        for df, sheet_name in zip(dfs, sheet_names):
            sheet = wb.sheets.add(sheet_name)
            sheet.range('A1').options(index=False).value = df
        wb.save(file_name)
        wb.close()

In [64]:
involved = clean_involved(df_raw)
additional_activities_hours, additional_activities_no_hours = clean_additional_activities(df_raw)
not_involved = clean_not_involved(df_raw)
optional_notes = clean_optional_notes(df_raw)

In [65]:
optional_notes = optional_notes[optional_notes['Optional Notes'] == optional_notes['Optional Notes']]

In [66]:
# List of DataFrames and corresponding sheet names
dfs = [optional_notes, not_involved, additional_activities_hours, additional_activities_no_hours, involved,]
sheet_names = ["Comments","Not Involved", "Additional - Hours", "Additional - No Hours", "Involved",]

# Export to Excel
export_to_excel('costing_out/cleaned_data_06102024.xlsx', dfs, sheet_names)

## adding onto the summary sheet

In [67]:
# def read_existing_sheets(file_path):
#     existing_data = {}
#     wb = xw.Book(file_path)
#     sheet_names = [sheet.name for sheet in wb.sheets]
#     for sheet_name in sheet_names:
#         sheet = wb.sheets[sheet_name]
#         existing_data[sheet_name] = sheet.range('A1').options(pd.DataFrame, expand='table').value
#     wb.close()
#     return existing_data, sheet_names

# def append_new_data(existing_data, new_data, sheet_names):
#     for sheet, new_df in zip(sheet_names, new_data):
#         if sheet in existing_data:
#             existing_df = existing_data[sheet]
#             if not existing_df.empty:
#                 # Check if columns match
#                 existing_columns = set(existing_df.columns)
#                 new_columns = set(new_df.columns)
#                 if existing_columns != new_columns:
#                     missing_in_new = existing_columns - new_columns
#                     extra_in_new = new_columns - existing_columns
#                     print(f"Column mismatch in sheet '{sheet}':")
#                     if missing_in_new:
#                         print(f"  Missing in new data: {missing_in_new}")
#                     if extra_in_new:
#                         print(f"  Extra in new data: {extra_in_new}")
#                     continue
#                 new_df = new_df.reindex(columns=existing_df.columns)
#                 updated_df = pd.concat([existing_df, new_df], ignore_index=True)
#             else:
#                 updated_df = new_df
#         else:
#             updated_df = new_df
#         existing_data[sheet] = updated_df
#     return existing_data

# def export_to_excel(new_file_path, data_dict):
#     full_path = os.path.abspath(new_file_path)
    
#     # Check if the file already exists and delete it if it does
#     if os.path.exists(full_path):
#         os.remove(full_path)
        
#     wb = xw.Book()  # Create a new workbook
#     for sheet_name, df in reversed(list(data_dict.items())):  # Reverse the order of sheets
#         if sheet_name in [sheet.name for sheet in wb.sheets]:
#             wb.sheets[sheet_name].clear()
#         else:
#             wb.sheets.add(sheet_name)
#         wb.sheets[sheet_name].range('A1').options(index=False).value = df
#     wb.save(new_file_path)
#     wb.close()

# def rename_sheets(file_path, new_sheet_names):
#     wb = xw.Book(file_path)
#     existing_sheet_names = [sheet.name for sheet in wb.sheets]
#     for old_name, new_name in zip(existing_sheet_names, new_sheet_names):
#         wb.sheets[old_name].name = new_name
#     wb.save()
#     wb.close()

# # Function to generate new sheet names with current date
# def generate_sheet_names(base_names):
#     current_date = datetime.now().strftime("%m.%d.%y")
#     new_sheet_names = [f"{name} ({current_date})" for name in base_names]
#     return new_sheet_names


In [68]:

# # List of DataFrames and corresponding sheet names
# # new_dfs = [optional_notes, not_involved, additional_activities_hours, additional_activities_no_hours, involved]
# new_dfs = [involved, not_involved, additional_activities_hours, additional_activities_no_hours, optional_notes]

# base_sheet_names = ["Involved", "Not Involved", "Additional- Hours", "Additional- No Hours", "Comments"]

# # Rename sheets in the existing Excel file
# existing_file_path = 'SITTMAT_050624_Summary.xlsx'

# # Generate new sheet names with current date
# new_sheet_names = generate_sheet_names(base_sheet_names)
# rename_sheets(existing_file_path, new_sheet_names)

# # Read existing Excel file
# existing_data, sheet_names = read_existing_sheets(existing_file_path)

# # Append new data to existing data
# updated_data = append_new_data(existing_data, new_dfs, sheet_names)

# # Export updated data to a new Excel file
# new_file_path = '/Users/liachin-purcell/work/sitt-matt/costing/python/SITTMAT_060624_Summary.xlsx'
# export_to_excel(new_file_path, updated_data)
